In [2]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
# Modelling
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor,AdaBoostRegressor
from sklearn.svm import SVR
from sklearn.linear_model import LinearRegression, Ridge,Lasso
from sklearn.metrics import r2_score, mean_absolute_error, mean_squared_error
from sklearn.model_selection import RandomizedSearchCV
from catboost import CatBoostRegressor
from xgboost import XGBRegressor
import warnings

In [3]:
df=pd.read_csv('stud.csv')

In [4]:
df.head()

,gender,race_ethnicity,parental_level_of_education,lunch,test_preparation_course,math_score,reading_score,writing_score
0,female,group B,bachelor's degree,standard,none,72,72,74
1,female,group C,some college,standard,completed,69,90,88
2,female,group B,master's degree,standard,none,90,95,93
3,male,group A,associate's degree,free/reduced,none,47,57,44
4,male,group C,some college,standard,none,76,78,75


In [5]:
X=df.drop('math_score',axis=1)
y=df['math_score']

In [6]:
# create column transformer with 3 types of transformers
num_features=X.select_dtypes(exclude='object').columns
cat_features=X.select_dtypes(include='object').columns
from sklearn.preprocessing import OneHotEncoder,StandardScaler
from sklearn.compose import ColumnTransformer
numeric_transformer=StandardScaler()
th_transformer=OneHotEncoder()
preprocessor=ColumnTransformer(
    [
        ("OneHotEncoder",th_transformer,cat_features),
        ('Standardscaler',numeric_transformer,num_features),
    ]
)


In [7]:
X=preprocessor.fit_transform(X)

In [8]:
X


array([[ 1.        ,  0.        ,  0.        , ...,  1.        ,
         0.19399858,  0.39149181],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         1.42747598,  1.31326868],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.77010859,  1.64247471],
       ...,
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.12547206, -0.20107904],
       [ 1.        ,  0.        ,  0.        , ...,  0.        ,
         0.60515772,  0.58901542],
       [ 1.        ,  0.        ,  0.        , ...,  1.        ,
         1.15336989,  1.18158627]])

In [9]:
#train_test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2,random_state=42)
X_train.shape,X_test.shape

((800, 19), (200, 19))

In [12]:
#create a evaluate function to give the all metrices
def evaluate_model(true,predicted):
    mae=mean_absolute_error(true,predicted)
    mse=mean_squared_error(true,predicted)
    rmse=np.sqrt(mean_squared_error(true,predicted))
    r2_square=r2_score(true,predicted)
    return mae,rmse,r2_square

In [15]:
models={
    "Linear Regression": LinearRegression(),
    "Lasso":Lasso(),
    "Ridge":Ridge(),
    "K-Neighbors Regressor":KNeighborsRegressor(),
    "Decision Tree":DecisionTreeRegressor(),
    "Random Forest Regressor":RandomForestRegressor(),
    "XGBRegressor":XGBRegressor(),
    "CatBoost Regressor":CatBoostRegressor(verbose=False),
    "AdaBoost Regressor":AdaBoostRegressor()
}
model_list=[]
r2_list=[]
for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #make prediction
    y_train_pred=model.predict(X_train)
    y_test_pred=model.predict(X_test)

    # Evaluate Train and test dataset
    model_train_mae,model_train_rmse,model_train_r2=evaluate_model(y_train,y_train_pred)
    model_test_mae,model_test_rmse,model_test_r2=evaluate_model(y_test,y_test_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])

    print('Model performance of Training set')
    print("Root mean square:{:.4f}".format(model_train_rmse))
    print("mean absolute error:{:.4f}".format(model_train_mae))
    print("r2_score:{:.4f}".format(model_train_r2))

    print('Model performance of test set')
    print("Root mean square:{:.4f}".format(model_test_rmse))
    print("mean absolute error:{:.4f}".format(model_test_mae))
    print("r2_score:{:.4f}".format(model_test_r2))
    r2_list.append(model_test_r2)
    
    print('='*35)
    print('\n')


Linear Regression
Model performance of Training set
Root mean square:5.3231
mean absolute error:4.2667
r2_score:0.8743
Model performance of test set
Root mean square:5.3940
mean absolute error:4.2148
r2_score:0.8804


Lasso
Model performance of Training set
Root mean square:6.5938
mean absolute error:5.2063
r2_score:0.8071
Model performance of test set
Root mean square:6.5197
mean absolute error:5.1579
r2_score:0.8253


Ridge
Model performance of Training set
Root mean square:5.3233
mean absolute error:4.2650
r2_score:0.8743
Model performance of test set
Root mean square:5.3904
mean absolute error:4.2111
r2_score:0.8806


K-Neighbors Regressor
Model performance of Training set
Root mean square:5.7077
mean absolute error:4.5167
r2_score:0.8555
Model performance of test set
Root mean square:7.2530
mean absolute error:5.6210
r2_score:0.7838


Decision Tree
Model performance of Training set
Root mean square:0.2795
mean absolute error:0.0187
r2_score:0.9997
Model performance of test set
Roo

In [18]:
pd.DataFrame(list(zip(model_list,r2_list)),columns=['Model Name','r2_score']).sort_values(by=["r2_score"],ascending=False)

,Model Name,r2_score
2,Ridge,0.880593
0,Linear Regression,0.880433
5,Random Forest Regressor,0.853830
8,AdaBoost Regressor,0.852429
7,CatBoost Regressor,0.851632
6,XGBRegressor,0.827797
1,Lasso,0.825320
3,K-Neighbors Regressor,0.783813
4,Decision Tree,0.728074
